## 8. Optimisation de portefeuille

Maintenant, utilisons l'optimiseur de portefeuille pour déterminer la meilleure allocation d'actifs.

In [ ]:
# Initialiser l'optimiseur de portefeuille
optimizer = PortfolioOptimizer(config)

# Nous allons utiliser les tickers communs aux données fondamentales et techniques
if 'combined_scores' in locals() and not combined_scores.empty:
    # Définir les paramètres d'optimisation
    risk_free_rate = float(config['PORTFOLIO_OPTIMIZATION']['risk_free_rate'])
    max_weight = float(config['PORTFOLIO_OPTIMIZATION']['max_weight_per_asset'])
    min_weight = float(config['PORTFOLIO_OPTIMIZATION']['min_weight_per_asset'])
    
    # Sélectionner les top N actions selon le score combiné
    top_n = 10  # Par exemple, les 10 meilleures actions
    top_tickers = combined_scores.sort_values(by='Combined_Score', ascending=False).index[:top_n]
    
    # Filtrer les données de prix pour les tickers sélectionnés
    top_price_data = clean_price_data[top_tickers]
    
    # Extraire les scores combinés pour les top tickers
    top_scores = combined_scores.loc[top_tickers, 'Combined_Score']
    
    print(f"Optimisation basée sur les {top_n} meilleures actions selon le score combiné
")
    
    # 1. Optimisation de Markowitz (ratio de Sharpe maximal)
    print("1. Exécution de l'optimisation de Markowitz...")
    weights_markowitz, exp_return_m, exp_risk_m, sharpe_m, details_m = optimizer.optimize_markowitz(
        price_data=top_price_data,
        target_return=None,  # None = maximisation du ratio de Sharpe
        risk_free_rate=risk_free_rate,
        max_weight=max_weight,
        min_weight=min_weight
    )
    
    # 2. Optimisation basée sur les scores
    print("
2. Exécution de l'optimisation basée sur les scores...")
    weights_score, exp_return_s, exp_risk_s, sharpe_s, details_s = optimizer.optimize_score_based(
        scores=top_scores,
        price_data=top_price_data,
        risk_free_rate=risk_free_rate,
        max_weight=max_weight,
        min_weight=min_weight
    )
    
    # 3. Optimisation par parité de risque
    print("
3. Exécution de l'optimisation par parité de risque...")
    weights_rp, exp_return_rp, exp_risk_rp, sharpe_rp, details_rp = optimizer.optimize_risk_parity(
        price_data=top_price_data,
        risk_free_rate=risk_free_rate,
        target_risk=None  # None = risque minimal
    )
    
    # Comparer les résultats des trois méthodes d'optimisation
    results = {
        'Markowitz': {
            'weights': weights_markowitz,
            'expected_return': exp_return_m,
            'expected_risk': exp_risk_m,
            'sharpe_ratio': sharpe_m
        },
        'Score-Based': {
            'weights': weights_score,
            'expected_return': exp_return_s,
            'expected_risk': exp_risk_s,
            'sharpe_ratio': sharpe_s
        },
        'Risk-Parity': {
            'weights': weights_rp,
            'expected_return': exp_return_rp,
            'expected_risk': exp_risk_rp,
            'sharpe_ratio': sharpe_rp
        }
    }
    
    # Afficher les résultats sous forme de tableau
    metrics_df = pd.DataFrame({
        'Rendement attendu (%)': [results[m]['expected_return'] * 100 for m in results],
        'Risque attendu (%)': [results[m]['expected_risk'] * 100 for m in results],
        'Ratio de Sharpe': [results[m]['sharpe_ratio'] for m in results]
    }, index=results.keys())
    
    print("
Comparaison des méthodes d'optimisation:")
    display(metrics_df)
    
    # Créer un DataFrame pour les poids
    weights_df = pd.DataFrame({
        'Markowitz': weights_markowitz,
        'Score-Based': weights_score,
        'Risk-Parity': weights_rp
    })
    
    # Ajouter les scores pour comparaison
    weights_df['Combined_Score'] = combined_scores.loc[weights_df.index, 'Combined_Score']
    
    print("
Allocations de portefeuille pour chaque méthode (%) et scores combinés:")
    display(weights_df.multiply(100).round(2))
else:
    print("Aucune donnée de score disponible pour l'optimisation de portefeuille.")


In [ ]:
# Visualiser les allocations de portefeuille
if 'weights_df' in locals() and not weights_df.empty:
    # Créer un graphique pour visualiser les allocations
    plt.figure(figsize=(15, 10))
    
    # Graphique à barres pour comparer les poids des différentes méthodes
    ax1 = plt.subplot(2, 1, 1)
    weights_df[['Markowitz', 'Score-Based', 'Risk-Parity']].plot(kind='bar', ax=ax1)
    plt.title('Comparaison des allocations de portefeuille par méthode', fontsize=14)
    plt.ylabel('Poids (%)', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, axis='y')
    plt.legend(title='Méthode')
    
    # Graphique en camembert pour la méthode basée sur les scores
    ax2 = plt.subplot(2, 2, 3)
    weights_score_sorted = weights_score.sort_values(ascending=False)
    weights_score_sorted.plot(kind='pie', autopct='%1.1f%%', ax=ax2)
    plt.title('Allocation Score-Based', fontsize=12)
    plt.ylabel('')  # Supprimer l'étiquette de l'axe y
    
    # Graphique du ratio rendement-risque
    ax3 = plt.subplot(2, 2, 4)
    methods = list(results.keys())
    returns = [results[m]['expected_return'] * 100 for m in methods]
    risks = [results[m]['expected_risk'] * 100 for m in methods]
    sharpes = [results[m]['sharpe_ratio'] for m in methods]
    
    # Utiliser la taille du marqueur pour représenter le ratio de Sharpe
    sizes = [s * 50 for s in sharpes]  # Ajuster la taille en fonction du ratio de Sharpe
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Couleurs pour chaque méthode
    
    scatter = ax3.scatter(risks, returns, s=sizes, c=colors, alpha=0.7)
    
    # Ajouter des étiquettes pour chaque point
    for i, method in enumerate(methods):
        ax3.annotate(method, (risks[i], returns[i]), 
                    xytext=(5, 5), textcoords='offset points')
    
    plt.title('Rendement vs Risque par méthode', fontsize=12)
    plt.xlabel('Risque (%)', fontsize=10)
    plt.ylabel('Rendement (%)', fontsize=10)
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("Aucune donnée d'allocation à visualiser.")


## 9. Backtesting du portefeuille

Réalisons un backtest du portefeuille optimisé pour évaluer sa performance historique.

In [ ]:
# Sélectionner une méthode d'optimisation pour le backtest
backtest_method = 'Score-Based'  # Choisir parmi 'Markowitz', 'Score-Based', 'Risk-Parity'

if 'results' in locals() and backtest_method in results:
    # Récupérer les poids de la méthode sélectionnée
    backtest_weights = results[backtest_method]['weights']
    
    # Définir la fréquence de rééquilibrage
    rebalance_freq = 'M'  # Mensuel
    
    print(f"Backtesting du portefeuille avec la méthode {backtest_method} et rééquilibrage {rebalance_freq}")
    
    # Exécuter le backtest
    performance, stats = optimizer.backtest_portfolio(
        price_data=top_price_data,
        weights=backtest_weights,
        rebalance_freq=rebalance_freq,
        risk_free_rate=risk_free_rate
    )
    
    # Créer un benchmark équipondéré pour comparer
    equal_weights = pd.Series(1/len(top_tickers), index=top_tickers)
    benchmark_performance, benchmark_stats = optimizer.backtest_portfolio(
        price_data=top_price_data,
        weights=equal_weights,
        rebalance_freq=rebalance_freq,
        risk_free_rate=risk_free_rate
    )
    
    # Fusionner les performances pour la comparaison
    comparison = pd.DataFrame({
        f'Portfolio {backtest_method}': performance['Portfolio_Value'],
        'Benchmark (Equal-Weight)': benchmark_performance['Portfolio_Value']
    })
    
    # Normaliser à 100 pour faciliter la comparaison
    comparison = 100 * comparison / comparison.iloc[0]
    
    # Afficher les statistiques de performance
    perf_stats = pd.DataFrame({
        f'Portfolio {backtest_method}': [
            stats['Cumulative_Return'] * 100,
            stats['Annual_Return'] * 100,
            stats['Annual_Volatility'] * 100,
            stats['Sharpe_Ratio'],
            stats['Max_Drawdown'] * 100,
            stats['Win_Rate'] * 100
        ],
        'Benchmark (Equal-Weight)': [
            benchmark_stats['Cumulative_Return'] * 100,
            benchmark_stats['Annual_Return'] * 100,
            benchmark_stats['Annual_Volatility'] * 100,
            benchmark_stats['Sharpe_Ratio'],
            benchmark_stats['Max_Drawdown'] * 100,
            benchmark_stats['Win_Rate'] * 100
        ]
    }, index=[
        'Rendement cumulatif (%)',
        'Rendement annualisé (%)',
        'Volatilité annualisée (%)',
        'Ratio de Sharpe',
        'Drawdown maximal (%)',
        'Taux de succès (%)'
    ])
    
    print("
Statistiques de performance:")
    display(perf_stats)
    
    # Créer un dashboard de performance
    dashboard = PerformanceDashboard(
        performance_data=comparison,
        portfolio_name=f'Portfolio {backtest_method}',
        benchmark_name='Benchmark (Equal-Weight)'
    )
    
    # Visualiser la performance
    plt.figure(figsize=(14, 8))
    plt.plot(comparison.index, comparison[f'Portfolio {backtest_method}'], label=f'Portfolio {backtest_method}')
    plt.plot(comparison.index, comparison['Benchmark (Equal-Weight)'], label='Benchmark (Equal-Weight)', linestyle='--')
    plt.title('Performance du portefeuille vs Benchmark', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Valeur (base 100)', fontsize=12)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Visualiser le drawdown
    dashboard.plot_drawdown()
else:
    print("Aucune donnée d'optimisation disponible pour le backtest.")


## 10. Conclusion et recommandations

D'après notre analyse, nous pouvons tirer plusieurs conclusions et formuler des recommandations d'investissement.

In [ ]:
# Récapituler les meilleures actions selon notre modèle
if 'combined_scores' in locals() and not combined_scores.empty:
    top_stocks = combined_scores.sort_values(by='Combined_Score', ascending=False).head(5)
    
    print("Top 5 des actions recommandées:
")
    for i, (ticker, row) in enumerate(top_stocks.iterrows(), 1):
        print(f"{i}. {ticker} - Score: {row['Combined_Score']:.4f}")
        print(f"   Score fondamental: {row['Fundamental_Score']:.4f}, Score technique: {row['Technical_Score']:.4f}")
    
    print("
Allocation de portefeuille optimale:
")
    if 'results' in locals():
        best_method = max(results, key=lambda x: results[x]['sharpe_ratio'])
        print(f"Méthode recommandée: {best_method} (Ratio de Sharpe: {results[best_method]['sharpe_ratio']:.4f})
")
        
        best_weights = results[best_method]['weights']
        for ticker, weight in best_weights.nlargest(10).items():
            print(f"{ticker}: {weight*100:.2f}%")
        
        print(f"
Rendement attendu: {results[best_method]['expected_return']*100:.2f}%")
        print(f"Risque attendu: {results[best_method]['expected_risk']*100:.2f}%")
    
    # Conseils généraux
    print("
Recommandations générales:
")
    print("1. Effectuer des analyses fondamentales approfondies avant d'investir")
    print("2. Diversifier le portefeuille selon les allocations optimales recommandées")
    print("3. Réévaluer les scores et rééquilibrer le portefeuille trimestriellement")
    print("4. Surveiller particulièrement les évolutions des scores techniques pour les ajustements tactiques")
    print("5. Compléter cette analyse quantitative par une analyse qualitative des entreprises")
else:
    print("Aucune donnée de score disponible pour formuler des recommandations.")
